**Table of contents**<a id='toc0_'></a>    
- [1. About the Notebook](#toc1_)    
- [2. Libraries and Packages](#toc2_)    
- [3. Connecting In Snowflake](#toc3_)    
- [4. Defining Functions](#toc4_)    
  - [4.1. Text Cleaning](#toc4_1_)    
  - [4.2. Tokenizing](#toc4_2_)    
  - [4.3. Remove Stopwords](#toc4_3_)    
  - [4.4. Count Words in each product and discard irrelevant products](#toc4_4_)    
  - [4.5. Count Vectorizer](#toc4_5_)    
  - [4.6. One Hot Encoding](#toc4_6_)    
  - [4.7. Recommendations with WALS](#toc4_7_)    
- [5. Loading Data and Applying all Functions](#toc5_)    
  - [5.1. Extracting list of all Main Categories](#toc5_1_)    
  - [5.2. Producing a Recommendation dataframe to each Main Category](#toc5_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[1. About the Notebook](#toc0_)

Notebook that creates feedback recommendations

# <a id='toc2_'></a>[2. Libraries and Packages](#toc0_)

In [9]:
import os
import snowflake.connector
import sqlalchemy
from loguru import logger
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report, silhouette_score
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import joblib

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/brunnokalyxton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/brunnokalyxton/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/brunnokalyxton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# <a id='toc3_'></a>[3. Connecting In Snowflake](#toc0_)

In [10]:
conn = snowflake.connector.connect(
    user='***********',
    password='***********',
    account='***********',
    warehouse='ANALYTICS_WH',
    database='AMAZON',
    schema='AMZ_DATA_GOLD',
    role = 'ANALYSTS'
)

# <a id='toc4_'></a>[4. Defining Functions](#toc0_)

In [11]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [12]:
def preprocess_text(text):
    words = word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(words)

# <a id='toc5_'></a>[5. Loading Category Data](#toc0_)

In [13]:
query1 = """
WITH CTE AS (
    SELECT 
        REV.REVIEWER_ID,
        REV.ASIN,
        REV.REVIEW_TEXT,
        PROD.MAIN_CATEGORY,
        AVG(REV.OVERALL) OVER (PARTITION BY REV.ASIN) AS "PRODUCT_AVG_RATING" 
    FROM
        PRODUCTS_REVIEWS AS REV
    INNER JOIN 
        PRODUCTS AS PROD ON REV.ASIN = PROD.ASIN
    WHERE 
        REV.REVIEW_TEXT IS NOT NULL AND 
        REV.REVIEW_TEXT <> '' 
)
SELECT 
    DISTINCT MAIN_CATEGORY
FROM 
    CTE
WHERE 
    PRODUCT_AVG_RATING >=4 AND
    MAIN_CATEGORY IS NOT NULL AND 
    MAIN_CATEGORY <> ''
GROUP BY
    1
"""

In [14]:
category = pd.read_sql_query(query1, conn)

/tmp/ipykernel_16979/3386730848.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  category = pd.read_sql_query(query1, conn)


In [15]:
category_list = category.MAIN_CATEGORY.values.tolist()

In [16]:
for i in category_list:
    query = """
    WITH CTE AS (
        SELECT 
            REV.ASIN,
            REV.REVIEWER_ID,
            REV.REVIEW_TEXT,
            REV.REVIEW_TIME,
            PROD.MAIN_CATEGORY,
            PROD.PRICE,
            AVG(REV.OVERALL) OVER (PARTITION BY REV.ASIN) AS "PRODUCT_AVG_RATING",
            (CASE 
                WHEN REV.OVERALL >= 4 THEN 'POSITIVE'
                ELSE 'NEGATIVE'
            END) AS SENTIMENT
        FROM 
            PRODUCTS_REVIEWS AS REV
        INNER JOIN 
            PRODUCTS AS PROD ON PROD.ASIN = REV.ASIN 
        WHERE 
            PROD.MAIN_CATEGORY = '{i}' AND 
            PROD.PRICE IS NOT NULL AND 
            REV.REVIEW_TEXT IS NOT NULL
    )
    SELECT 
        *
    FROM 
        (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY REVIEWER_ID ORDER BY REVIEW_TIME DESC) AS ROW_NUMBER
    FROM
        CTE
    )
    WHERE 
        ROW_NUMBER = 1 AND 
        PRODUCT_AVG_RATING <= 2 
    """.format(i=i)
    
    logger.info('Loading Data category {i}...'.format(i=i))
    reviews = pd.read_sql_query(query, conn)

    if len(reviews) > 100:

        reviews['clean_reviews'] = reviews['REVIEW_TEXT'].apply(preprocess_text)

        X = reviews['clean_reviews']
        y = reviews['SENTIMENT']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Feature extraction using TF-IDF
        vectorizer = TfidfVectorizer()
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        # Training a Support Vector Machine (SVM) classifier
        logger.info('Training Classifier Model {i}...'.format(i=i))
        classifier = SVC(kernel='linear')
        classifier.fit(X_train_tfidf, y_train)

        # Predicting on the test set
        y_pred = classifier.predict(X_test_tfidf)

        # Evaluating the model
        accuracy = accuracy_score(y_test, y_pred)

        reviews['predicted_sentiments'] = classifier.predict(vectorizer.transform(reviews['clean_reviews']))
        negative_reviews = reviews[reviews['predicted_sentiments'] == 'NEGATIVE']

        logger.info('Creating Clusters for {i}...'.format(i=i))
        silhouette_scores = []
        k_values = list(range(3,10))  # Maximum number of clusters to try

        for k in k_values:
            kmeans = KMeans(n_clusters=k, random_state=42)
            kmeans.fit(vectorizer.transform(negative_reviews['clean_reviews']))
            silhouette_scores.append(silhouette_score(vectorizer.transform(negative_reviews['clean_reviews']), kmeans.labels_))

        silhouette_results = dict(zip(k_values, silhouette_scores))
        maxSilhouette = dict(sorted(silhouette_results.items(), key=lambda item: item[1], reverse=True))
        k = list(maxSilhouette.keys())[0] # optimum number of clusters

        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(vectorizer.transform(negative_reviews['clean_reviews']))
        negative_reviews.loc[:, 'cluster'] = kmeans.labels_

        logger.info('Saving Complaints {i}...'.format(i=i))
        # Print the grouped complaints for each cluster
        for j in range(k):
            df_name = f'cluster_complaints_{i}_{j}'
            df_name = negative_reviews[negative_reviews['cluster'] == j][['REVIEWER_ID', 'REVIEW_TEXT']]
            if not os.path.exists(f"../output/complaints/{i}"):
                os.makedirs(f"../output/complaints/{i}")
            df_name.to_parquet(
                        path="../output/complaints/{i}/cluster_complaints_{i}_{j}.parquet".format(i=i, j=j),
                        engine="auto"
                    ) 
            # print(f"\nCluster {j + 1} - Complaints:")
            # print(negative_reviews[negative_reviews['cluster'] == j]['REVIEW_TEXT'])

        logger.info('Savind Wordclouds {i}...'.format(i=i))
        for j in range(k):
            cluster_reviews = negative_reviews[negative_reviews['cluster'] == j]['clean_reviews']
            all_text = ' '.join(cluster_reviews)
            
            wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(all_text)
            
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis("off")
            plt.title(f"{i}- Cluster {j + 1}")
            if not os.path.exists(f"../output/word_clouds/{i}"):
                os.makedirs(f"../output/word_clouds/{i}")
        
            # Save the image to the folder
            image_filename = f"../output/word_clouds/{i}/word_cloud_cluster_{i}_{j + 1}.png"
            plt.savefig(image_filename)
            plt.clf()

        logger.info('Saving Models {i}...'.format(i=i))
        # Save the trained classifier and vectorizer to files
        joblib.dump(classifier, '../output/models/complaint_classifier_model_{i}.pkl'.format(i=i))
        joblib.dump(vectorizer, '../output/models/tfidf_vectorizer_{i}.pkl'.format(i=i))
    else:
        print(f"Size for category '{i}' is 100".format(i=i))
        pass

2023-08-05 12:05:55.899 | INFO     | __main__:<module>:40 - Loading Data category Movies & Tv...
/tmp/ipykernel_16979/1170871101.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews = pd.read_sql_query(query, conn)
2023-08-05 12:06:03.606 | INFO     | __main__:<module>:57 - Training Classifier Model Movies & Tv...
2023-08-05 12:06:09.120 | INFO     | __main__:<module>:70 - Creating Clusters for Movies & Tv...
/tmp/ipykernel_16979/1170871101.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_reviews.loc[:, 'cluster'] = kmeans.labels_
2023-08-05 12:06:35.577 | INFO     | __ma

Size for category 'Alexa Skills' is 100


/tmp/ipykernel_16979/1170871101.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews = pd.read_sql_query(query, conn)
2023-08-05 12:06:43.794 | INFO     | __main__:<module>:40 - Loading Data category Portable Audio & Accessories...


Size for category 'Audible Audiobooks' is 100


/tmp/ipykernel_16979/1170871101.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews = pd.read_sql_query(query, conn)
2023-08-05 12:06:44.468 | INFO     | __main__:<module>:57 - Training Classifier Model Portable Audio & Accessories...
2023-08-05 12:06:44.480 | INFO     | __main__:<module>:70 - Creating Clusters for Portable Audio & Accessories...
/tmp/ipykernel_16979/1170871101.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_reviews.loc[:, 'cluster'] = kmeans.labels_
2023-08-05 12:06:45.118 | INFO     | __main__:<module>:87 - Saving Complaints Portable Audio & Accessori

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>